In [332]:
#BeautifulSoup
from bs4 import BeautifulSoup
import requests

import re
import pandas as pd
import numpy as np

In [333]:
url = "https://www.paruvendu.fr/immobilier/annonceimmofo/liste/listeAnnonces?tt=1&at=1&nbp0=99&pa=FR&lo=75,77,78,91,92,93,94,95&lol=0&ray=50"

In [ ]:
list_surface=[]
for elt in list_bien:
    b=elt.split(",")
    if len(b) >1:
        c=b[1].split()
        list_surface.append(c[0])
    #c=b[1].split()
    #print(c[0])
    
    

list_surface    


In [334]:
all_df = []

for page in range(1,501):
    
    req = requests.get(url + "&p=" + str(page))
    soup = BeautifulSoup(req.text, "html.parser")
    
    price = [price.get_text() for price in soup.find_all("div", class_='ergov3-priceannonce')]
    good_prices = [prices.strip().replace("\r\n*", "") for prices in price]
    
    cities = [city.get_text() for city in soup.find_all("cite")]
    good_cities = [cite.strip().replace("\r\n*", "") for cite in cities]
    
    annonceur = [anno.get_text() for anno in soup.find_all("div", class_='ergov3-bottomannonce ergov3-bottomannonce_2lines')]
    good_annonceur = [annonc.strip().replace("\r\n*", "") for annonc in annonceur]
    
    text_annonce = [annonce.get_text() for annonce in soup.find_all("p", class_='txt-long')]
    good_text_annonce = [gta.strip().replace("\r\n*", "") for gta in text_annonce]
    
    title_liste = []
    for title in soup.select('div.ergov3-h3'):
        title_liste.append(title.span.text)
    good_title_liste = [gtl.strip().replace("\s+", " ") for gtl in title_liste]
    
    all_df.append(pd.DataFrame({'Title': good_title_liste, 'Prix': good_prices,'Ville': good_cities, 'Annonceur': good_annonceur, 'Description': good_text_annonce}))
    

out = pd.concat(all_df, ignore_index = True)

In [335]:
out.head()

,Title,Prix,Ville,Annonceur,Description
0,"Appartement\r\n3\r\npièces,\t\t\t\t\t\t\t\t\t\...",348 000 €,Noisy-le-Grand (93160),Professionnel\n\n\n\nAGENCE DE L'ANCIENNE MAIR...,Appartement noisy le grand 3 pièce(s) 63.17 m2...
1,"Maison\r\n8\r\npièces,\t\t\t\t\t\t\t\t\t\t\t\t...",315 000 €,Ablis (78660),Professionnel\n\n\n\nAGENCE DE L'OCRE\r\n41 an...,Maison à 10' ablis sous sol 5 chambres. a 10' ...
2,"Appartement\r\n3\r\npièces,\t\t\t\t\t\t\t\t\t\...",340 000 €,Alfortville (94140),Professionnel\n\n\n\nADJ GESTION\r\n43 annonce...,A vendre appartement 3 pièces de 56.76 m² alf...
3,"Appartement\r\n4\r\npièces,\t\t\t\t\t\t\t\t\t\...",1 299 000 €,Paris 16,Professionnel\n\n\n\nL'ADRESSE PARIS 16E\r\n23...,Ranelagh - appartement 4p 90m². au 5ème étage ...
4,"Maison\r\n5\r\npièces,\t\t\t\t\t\t\t\t\t\t\t\t...",575 000 €,Fontenay-sous-Bois (94120),Professionnel\n\n\n\nIMMOGEF FONTENAY\r\n21 an...,Maison fontenay sous bois 5 pièce(s) 118.50 m2...


In [336]:
out = out.replace(r'\r+|\n+|\t+',' ', regex = True)

In [337]:
out.head()

,Title,Prix,Ville,Annonceur,Description
0,"Appartement 3 pièces, 63 m²",348 000 €,Noisy-le-Grand (93160),Professionnel AGENCE DE L'ANCIENNE MAIRIE 7 a...,Appartement noisy le grand 3 pièce(s) 63.17 m2...
1,"Maison 8 pièces, 156 m²",315 000 €,Ablis (78660),Professionnel AGENCE DE L'OCRE 41 annonces V...,Maison à 10' ablis sous sol 5 chambres. a 10' ...
2,"Appartement 3 pièces, 57 m²",340 000 €,Alfortville (94140),Professionnel ADJ GESTION 43 annonces Voir l...,A vendre appartement 3 pièces de 56.76 m² alf...
3,"Appartement 4 pièces, 90 m²",1 299 000 €,Paris 16,Professionnel L'ADRESSE PARIS 16E 23 annonces...,Ranelagh - appartement 4p 90m². au 5ème étage ...
4,"Maison 5 pièces, 119 m²",575 000 €,Fontenay-sous-Bois (94120),Professionnel IMMOGEF FONTENAY 21 annonces V...,Maison fontenay sous bois 5 pièce(s) 118.50 m2...


In [338]:
#out['Bien'] = out.Title.str.extract(r'(^[a-zA-Z]+)', expand=True)
out['Bien'] = out.Title.str.extract(r'(\S+)', expand=True)
out["Pièces"] = out["Title"].str.extract(r'(\d+)\s*pièces?', expand=False)
out['Taille'] = out["Title"].str.extract(r'(\d+(?:\.\d+)?)\s*m²', expand=False)
out["Annonceur"] = out["Annonceur"].str.extract(r'(^[a-zA-Z]+)', expand=False)
out["Prix"] = out["Prix"].replace(r'€','', regex = True)
out["Code Postal"] = out["Ville"].str.extract(r'(\d+)', expand=False)
#out["Ville"] = out["Ville"].str.extract(r'(.*)\s\(\d+\)', expand=False) #|
out["Ville"] = out["Ville"].str.extract(r'([^0-9]+)\s\(\d+\)', expand=False)

In [339]:
out.head()

,Title,Prix,Ville,Annonceur,Description,Bien,Pièces,Taille,Code Postal
0,"Appartement 3 pièces, 63 m²",348 000,Noisy-le-Grand,Professionnel,Appartement noisy le grand 3 pièce(s) 63.17 m2...,Appartement,3,63,93160
1,"Maison 8 pièces, 156 m²",315 000,Ablis,Professionnel,Maison à 10' ablis sous sol 5 chambres. a 10' ...,Maison,8,156,78660
2,"Appartement 3 pièces, 57 m²",340 000,Alfortville,Professionnel,A vendre appartement 3 pièces de 56.76 m² alf...,Appartement,3,57,94140
3,"Appartement 4 pièces, 90 m²",1 299 000,NaN,Professionnel,Ranelagh - appartement 4p 90m². au 5ème étage ...,Appartement,4,90,16
4,"Maison 5 pièces, 119 m²",575 000,Fontenay-sous-Bois,Professionnel,Maison fontenay sous bois 5 pièce(s) 118.50 m2...,Maison,5,119,94120


In [340]:
#remove ventes
out = out[out["Bien"].str.contains("Vente") == False]

In [341]:
new_df = out[['Bien', 'Pièces', 'Taille', 'Prix', 'Ville', 'Code Postal', 'Annonceur', 'Description']]

In [342]:
new_df.head()

,Bien,Pièces,Taille,Prix,Ville,Code Postal,Annonceur,Description
0,Appartement,3,63,348 000,Noisy-le-Grand,93160,Professionnel,Appartement noisy le grand 3 pièce(s) 63.17 m2...
1,Maison,8,156,315 000,Ablis,78660,Professionnel,Maison à 10' ablis sous sol 5 chambres. a 10' ...
2,Appartement,3,57,340 000,Alfortville,94140,Professionnel,A vendre appartement 3 pièces de 56.76 m² alf...
3,Appartement,4,90,1 299 000,NaN,16,Professionnel,Ranelagh - appartement 4p 90m². au 5ème étage ...
4,Maison,5,119,575 000,Fontenay-sous-Bois,94120,Professionnel,Maison fontenay sous bois 5 pièce(s) 118.50 m2...


In [343]:
#effacer les NaN
new_df.dropna() #fillna("")

,Bien,Pièces,Taille,Prix,Ville,Code Postal,Annonceur,Description
0,Appartement,3,63,348 000,Noisy-le-Grand,93160,Professionnel,Appartement noisy le grand 3 pièce(s) 63.17 m2...
1,Maison,8,156,315 000,Ablis,78660,Professionnel,Maison à 10' ablis sous sol 5 chambres. a 10' ...
2,Appartement,3,57,340 000,Alfortville,94140,Professionnel,A vendre appartement 3 pièces de 56.76 m² alf...
4,Maison,5,119,575 000,Fontenay-sous-Bois,94120,Professionnel,Maison fontenay sous bois 5 pièce(s) 118.50 m2...
5,Appartement,4,80,207 000,Chennevières-sur-Marne,94430,Professionnel,Appartement |chennevières sur marne | 3 chambr...
...,...,...,...,...,...,...,...,...
13495,Appartement,3,64,460 000,Asnières-sur-Seine,92600,Professionnel,Vente appartement 3 pièces. iad france - samu...
13496,Maison,4,112,330 000,Le Blanc-Mesnil,93150,Professionnel,Vente maison/villa 4 pièces. iad france - son...
13497,Maison,3,84,286 000,Carrières-sous-Poissy,78955,Professionnel,Vente maison/villa 3 pièces. iad france - syl...
13498,Appartement,3,63,425 000,Ivry-sur-Seine,94200,Professionnel,Vente appartement 3 pièces. iad france - chri...


In [344]:
new_df=new_df.replace([np.inf, -np.inf], np.nan)

In [ ]:

new_df.head()

In [348]:
new_df=new_df.replace([np.inf, -np.inf], np.nan)
new_df=new_df.replace("NC","")
new_df.dropna()
new_df= new_df.dropna()
new_df.isna().sum()

#new_df["Taille"]=new_df["Taille"].astype(int)
#new_df.dropna()


Bien           0
Pièces         0
Taille         0
Prix           0
Ville          0
Code Postal    0
Annonceur      0
Description    0
dtype: int64

In [349]:
new_df["Taille"]=new_df["Taille"].apply(pd.to_numeric)

In [350]:
new_df["Pièces"]=new_df["Pièces"].apply(pd.to_numeric)

In [351]:
new_df.dtypes

Bien           object
Pièces          int64
Taille          int64
Prix           object
Ville          object
Code Postal    object
Annonceur      object
Description    object
dtype: object

In [352]:
new_df.drop_duplicates().reset_index(drop=True)
new_df.duplicated()

0        False
1        False
2        False
4        False
5        False
         ...  
13495    False
13496    False
13497    False
13498    False
13499    False
Length: 11183, dtype: bool

In [353]:
bien_col = ['Pièces','Taille','Prix','Description']

In [354]:
bien_df = new_df[bien_col]

In [355]:
bien_df.head()

,Pièces,Taille,Prix,Description
0,3,63,348 000,Appartement noisy le grand 3 pièce(s) 63.17 m2...
1,8,156,315 000,Maison à 10' ablis sous sol 5 chambres. a 10' ...
2,3,57,340 000,A vendre appartement 3 pièces de 56.76 m² alf...
4,5,119,575 000,Maison fontenay sous bois 5 pièce(s) 118.50 m2...
5,4,80,207 000,Appartement |chennevières sur marne | 3 chambr...


In [356]:
type_df = new_df[["Bien"]]

In [357]:
type(type_df)

pandas.core.frame.DataFrame

In [358]:
type_df.head()

,Bien
0,Appartement
1,Maison
2,Appartement
4,Maison
5,Appartement


In [359]:
loc_col = ['Ville','Code Postal']

In [360]:
loc_df = new_df[loc_col]
type(loc_df)

pandas.core.frame.DataFrame

In [361]:
loc_df=loc_df.rename({'ville':'Ville', 'Code Postal': 'Code_Postal'}, axis=1)

In [362]:
loc_df.head()

,Ville,Code_Postal
0,Noisy-le-Grand,93160
1,Ablis,78660
2,Alfortville,94140
4,Fontenay-sous-Bois,94120
5,Chennevières-sur-Marne,94430


In [363]:
ann_df = new_df[["Annonceur"]]

In [364]:
ann_df.head()

,Annonceur
0,Professionnel
1,Professionnel
2,Professionnel
4,Professionnel
5,Professionnel


In [132]:
import mysql.connector as mysqlConnector

In [133]:
# Se Connecter à la BDD paruvendu
#TBD
try:
    conn = mysqlConnector.connect(host='Localhost',user='root',passwd='toor', database='paruvendu')
    conn.autocommit = True
    if conn:
        print("Connection Successful!")
    else:
        print("Connection Failed!")
    cur = conn.cursor()
        # Afficher la version de mySQL 
    cur.execute("SELECT version();")
    version = cur.fetchone()
    print("Version : ", version,"\n")
except (Exception, mysqlConnector.Error) as error :
    print ("Erreur lors de la connexion à mySQL", error)


Connection Successful!
Version :  ('8.0.29',) 



In [134]:
sql = '''
        DROP TABLE if exists biens cascade;
        CREATE TABLE biens (
        biens_id SERIAL PRIMARY KEY NOT NULL,
        Pièces int ,
        Taille int,
        Prix float,
        Description text
        
      ); 
      DROP TABLE if exists localisation cascade;
      CREATE TABLE localisation (
          loc_id SERIAL PRIMARY KEY NOT NULL,
          Ville varchar(255),
          Code_Postal int
        
      ); 
      
        DROP TABLE if exists typebien cascade;
        CREATE TABLE typebien (
        type_id SERIAL PRIMARY KEY NOT NULL,
        type_bien varchar(255)
        
      ); 
      DROP TABLE if exists annonceur cascade;
        CREATE TABLE annonceur (
        ann_id SERIAL PRIMARY KEY NOT NULL,
        type_ann varchar(255)
        );
        '''

In [365]:
bien_tup = list(set([tuple(x) for x in bien_df.to_numpy()]))
bien_tup

[(3,
  64,
  '299 980 ',
  'Appartement de 3 pièces principales. a 8 mn à pied de la gare, appartement de 3 pièces : séjour sur balcon , cuisine équipée avec cellier, 2 chambres, salle de bain, 1 place de parking, 1 cave.  vue dégagée.'),
 (4,
  69,
  '215 000 ',
  'Vente appartement 4 pièces. iad france  - sylvaine brousse () vous propose : les clayes-sous-bois, centre-ville \x96  au 3ème et dernier étage sans ascenseur, proche de toutes les commodités. je vous propose cet appartement de 4 pièces, parfaitement entretenu, comprenant une cuisine...'),
 (8,
  160,
  '565 000 ',
  "Maison 160 m2 - 5 chambres - dépendance - garage - jardin. idéalement située, proche centre-ville, grande maison familiale d'une surface totale de 190 m2 édifiée sur 3 niveaux :    au rdc, grand hall d'entrée, buanderie, garage réaménagé en salle de sport, 2 chambres et salle d'eau avec..."),
 (14,
  401,
  '1 200 000 ',
  'Maison. thibault woirgard, efficity immobilier vous propose, à seulement 45 min de paris

In [135]:
 with conn.cursor() as cur:
        cur.execute(sql)

In [136]:
bien_column=list(bien_df.columns)
bien_column

['Pièces', 'Taille', 'Prix', 'Description']

In [289]:
try:
    conn = mysqlConnector.connect(host='Localhost',user='root',passwd='toor', database='paruvendu')
    conn.autocommit = True
    if conn:
        print("Connection Successful!")
    else:
        print("Connection Failed!")
        
except (Exception, mysqlConnector.Error) as error :
    print ("Erreur lors de la connexion à mySQL", error)

    

Connection Successful!


In [138]:
for elem in bien_tup:
    elems = "INSERT INTO biens("+bien_column[0]+","+bien_column[1]+","+bien_column[2]+","+bien_column[3]+") VALUES(%s, %s, %s, %s)"
    quer=(elem[0],elem[1],elem[2],elem[3])
    #print(quer)
    #print(elems)
    with conn.cursor() as cur:
        cur.execute(elems,quer)
        conn.autocommit = True

In [207]:
loc_column=list(loc_df.columns)
loc_column

['Ville', 'Code_Postal']

In [208]:
loc_tup = list(set([tuple(x) for x in loc_df.to_numpy()]))
loc_tup

[('Breuil-Bois-Robert', '78930'),
 ('Saintry-sur-Seine', '91250'),
 ('Gommecourt', '78270'),
 ('Remauville', '77710'),
 ('Villejust', '91140'),
 ('Bray-et-Lû', '95710'),
 ('Vaux-le-Pénil', '77000'),
 ('Orly', '94310'),
 ('Pézarches', '77131'),
 ('Varennes-sur-Seine', '77130'),
 ('La Rochette', '77000'),
 ('Persan', '95340'),
 ('Ville-Saint-Jacques', '77130'),
 ('Les Alluets-le-Roi', '78580'),
 ('Chalmaison', '77650'),
 ('Écharcon', '91540'),
 ('Achères', '78260'),
 ('Les Pavillons-sous-Bois', '93320'),
 ('Orcemont', '78125'),
 ('Champigny-sur-Marne', '94500'),
 ('Neuilly-Plaisance', '93360'),
 ('Villeneuve-sur-Bellot', '77510'),
 ('Saint-Chéron', '91530'),
 ('Buthiers', '77760'),
 ('Saint-Loup-de-Naud', '77650'),
 ('Bussy-Saint-Georges', '77600'),
 ('Champeaux', '77720'),
 ('Marcoussis', '91460'),
 ('Paley', '77710'),
 ('Saint-Germain-sous-Doue', '77169'),
 ('hondevilliers', '77'),
 ('Couilly-Pont-aux-Dames', '77860'),
 ('Saint-Germain-Laxis', '77950'),
 ('Égreville', '77620'),
 ('Choi

req_unicode='''ALTER DATABASE paruvendu CHARACTER SET = utf8mb4 COLLATE = utf8mb4 unicode_ci;
               ALTER TABLE Ville CONVERT TO CHARACTER SET utf8mb4 COLLATE utf8mb4 unicode ci;'''

In [209]:
req_unicode=''' ALTER TABLE localisation MODIFY Ville varchar(255) CHARACTER SET utf8'''

In [210]:
with conn.cursor() as cur:
        cur.execute(req_unicode)

In [212]:
for elem in loc_tup:
    elems = "INSERT INTO localisation("+loc_column[0]+","+loc_column[1]+") VALUES(%s, %s)"
    quer=(elem[0],elem[1])
    print(quer)
    print(elems)
    with conn.cursor() as cur:
        cur.execute(elems,quer)
        #conn.autocommit = True

('Breuil-Bois-Robert', '78930')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Saintry-sur-Seine', '91250')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Gommecourt', '78270')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Remauville', '77710')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Villejust', '91140')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Bray-et-Lû', '95710')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Vaux-le-Pénil', '77000')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Orly', '94310')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Pézarches', '77131')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Varennes-sur-Seine', '77130')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('La Rochette', '77000')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Persan', '95340')
INSERT INTO localisation(Ville,Code_Postal) VALU

('Vélizy-Villacoublay', '78140')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Arpajon', '91290')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Survilliers', '95470')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Le Coudray-Montceaux', '91830')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Villemaréchal', '77710')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Épiais-Rhus', '95810')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Villeneuve-le-Roi', '94290')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Saint-Soupplets', '77165')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('La Grande-Paroisse', '77130')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Magny-en-Vexin', '95420')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Bassevelle', '77750')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Le Vésinet', '78110')
INSERT INTO locali

('Neuilly-sur-Marne', '93330')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Limay', '78520')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Maison-Rouge', '77370')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Oinville-sur-Montcient', '78250')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Neufmoutiers-en-Brie', '77610')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Antony', '92160')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Combs-la-Ville', '77380')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Coupvray', '77700')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Bois-le-Roi', '77590')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Les Ormes-sur-Voulzie', '77134')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Chaussy', '95710')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Pamfou', '77830')
INSERT INTO localisation(Ville,Cod

('Villecerf', '77250')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Saint-Denis', '93200')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Rungis', '94150')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Montigny-Lencoup', '77520')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Pringy', '77310')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Chartronges', '77320')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Orphin', '78125')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Maisse', '91720')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Ormoy', '91540')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Mormant', '77720')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Orgeval', '78630')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Grez-sur-Loing', '77880')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Isles-lès-Villenoy

('Le Tremblay-sur-Mauldre', '78490')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Aigremont', '78240')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Guernes', '78520')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Bougival', '78380')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Coulommiers', '77120')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Chauconin-Neufmontiers', '77124')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Vinantes', '77230')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Mulcent', '78790')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Presles', '95590')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Villeroy', '77410')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Le Mesnil-Saint-Denis', '78320')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Le Pecq', '78230')
INSERT INTO localisation(Ville,Code_Postal) V

('Meudon La Foret', '92360')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Les Chapelles-Bourbon', '77610')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Nangis', '77370')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Marolles-sur-Seine', '77130')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Champs-sur-Marne', '77420')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Montry', '77450')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Jouars-Pontchartrain', '78760')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Butry-sur-Oise', '95430')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('La Chapelle-Gauthier', '77720')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Meulan', '78250')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Gonesse', '95500')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Viry-Châtillon', '91170')
INSERT INTO localisa

('Menucourt', '95180')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Chevry-en-Sereine', '77710')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Oncy-sur-École', '91490')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Brou-sur-Chantereine', '77177')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Juvisy-sur-Orge', '91260')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Champagne-sur-Seine', '77430')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Ecquevilly', '78920')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Montereau-sur-le-Jard', '77950')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Montreuil-sur-Epte', '95770')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Coignières', '78310')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Bezons', '95870')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Hardricourt', '78250')
INSERT INTO l

('Villeron', '95380')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('La Ferté-Gaucher', '77320')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Saint-Mammès', '77670')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Saint-Germain-sur-Morin', '77860')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Yèbles', '77390')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Maulette', '78550')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Gouvernes', '77400')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Charny', '77410')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Villemareuil', '77470')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Méréville', '91660')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Châtenay-sur-Seine', '77126')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Orsay', '91400')
INSERT INTO localisation(Ville,Code_Postal) VALUES(

('Viarmes', '95270')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Frépillon', '95740')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Nainville-les-Roches', '91750')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Marines', '95640')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Meilleray', '77320')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Nozay', '91620')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Argenteuil', '95100')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Guigneville-sur-Essonne', '91590')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('La Ferté-Alais', '91590')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Fontenay-le-Fleury', '78330')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Donnemarie-Dontilly', '77520')
INSERT INTO localisation(Ville,Code_Postal) VALUES(%s, %s)
('Argentières', '77390')
INSERT INTO localisation(Ville,

In [214]:
type_df.value_counts

<bound method DataFrame.value_counts of               Bien
0           Maison
1      Appartement
2      Appartement
3      Appartement
4      Appartement
...            ...
13494  Appartement
13495       Maison
13497  Appartement
13498  Appartement
13499  Appartement

[11229 rows x 1 columns]>

In [216]:
type_df=type_df.rename({'Bien':'type_bien'}, axis=1)

In [226]:
type_df.value_counts()

type_bien        
Appartement          6066
Maison               4960
Duplex/triplex         76
Villa                  70
Propriété/château      34
Loft                   15
Chalet                  4
Atelier                 2
Ferme                   1
Moulin                  1
dtype: int64

In [232]:
typeb=type_df["type_bien"].unique()

In [233]:
for el in typeb:
    print(el)

Maison
Appartement
Loft
Villa
Duplex/triplex
Propriété/château
Atelier
Chalet
Moulin
Ferme


In [284]:
type_df.head()

,type_bien
0,Maison
1,Appartement
2,Appartement
3,Appartement
4,Appartement


In [285]:
type_df['type_bien'].unique()

array(['Maison', 'Appartement', 'Loft', 'Villa', 'Duplex/triplex',
       'Propriété/château', 'Atelier', 'Chalet', 'Moulin', 'Ferme'],
      dtype=object)

In [235]:
type_df_column=list(type_df.columns)
type_df_column

['type_bien']

In [242]:
di=type_df['type_bien'].reset_index(level=None, drop=True, inplace=False)
dico={}
#dico{0= "appartement", 1="maison"}
dico[0]="Maison"
dico[1]="Appartement"


for e, el in enumerate(dico.values()):
    print(e,el)

0 Maison
1 Appartement


In [236]:
type_tup = list(set([tuple(x) for x in type_df.to_numpy()]))
type_tup

[('Loft',),
 ('Moulin',),
 ('Propriété/château',),
 ('Ferme',),
 ('Villa',),
 ('Appartement',),
 ('Maison',),
 ('Duplex/triplex',),
 ('Chalet',),
 ('Atelier',)]

In [237]:
for el in type_tup:
    with conn.cursor() as cur:
        cur.execute("INSERT INTO typebien(type_bien) VALUES(%s)", (el))

conn.commit()

In [238]:
ann_df.value_counts()

Annonceur    
Professionnel    10963
Particulier        266
dtype: int64

In [239]:
ann_df.head()

,Annonceur
0,Professionnel
1,Professionnel
2,Professionnel
3,Professionnel
4,Professionnel


In [240]:
ann_tup = list(set([tuple(x) for x in ann_df.to_numpy()]))
ann_tup

[('Professionnel',), ('Particulier',)]

In [241]:
for el in ann_tup:
    with conn.cursor() as cur:
        cur.execute("INSERT INTO annonceur(type_ann) VALUES(%s)", (el))

conn.commit()

In [269]:
bien_df.head()

,Pièces,Taille,Prix,Description
0,4,125,386550.0,Maison gagny 4 pièces 125 m2. dans un secteur ...
1,2,39,284000.0,Appartement 2 pièces de 38m² environ à vendre...
2,3,69,399000.0,Appartement 3 pièces de 69 m² environ. votre ...
3,3,70,486000.0,Appartement le plessis robinson 3 pièces 70 m2...
4,3,61,330000.0,Appartement à vendre le plessis robinson 3 pi...


In [270]:
loc_df.head()

,Ville,Code_Postal
0,Gagny,93220
1,Clamart,92140
2,Clamart,92140
3,Le Plessis-Robinson,92350
4,Le Plessis-Robinson,92350


req_fk=''' ALTER TABLE biens
           ADD COLUMN loc_id int,
           ADD COLUMN type_id int,
           ADD COLUMN ann_id int,
           ADD FOREIGN KEY (loc_id)
                REFERENCES localisation (loc_id)
                ON UPDATE CASCADE ON DELETE CASCADE,
           ADD FOREIGN KEY (type_id)
                REFERENCES typebien (type_id)
                ON UPDATE CASCADE ON DELETE CASCADE,
            ADD FOREIGN KEY (ann_id)
                REFERENCES Annonceur (ann_id)
                ON UPDATE CASCADE ON DELETE CASCADE;'''

In [294]:
type_df.head(100)

,type_bien
0,Maison
1,Appartement
2,Appartement
3,Appartement
4,Appartement
...,...
115,Maison
116,Appartement
117,Appartement
118,Maison


In [273]:
bien_id = []
for i in range(1,len(new_df['Bien'])+1):
    print(i)
    bien_id.append(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143


3798
3799
3800
3801
3802
3803
3804
3805
3806
3807
3808
3809
3810
3811
3812
3813
3814
3815
3816
3817
3818
3819
3820
3821
3822
3823
3824
3825
3826
3827
3828
3829
3830
3831
3832
3833
3834
3835
3836
3837
3838
3839
3840
3841
3842
3843
3844
3845
3846
3847
3848
3849
3850
3851
3852
3853
3854
3855
3856
3857
3858
3859
3860
3861
3862
3863
3864
3865
3866
3867
3868
3869
3870
3871
3872
3873
3874
3875
3876
3877
3878
3879
3880
3881
3882
3883
3884
3885
3886
3887
3888
3889
3890
3891
3892
3893
3894
3895
3896
3897
3898
3899
3900
3901
3902
3903
3904
3905
3906
3907
3908
3909
3910
3911
3912
3913
3914
3915
3916
3917
3918
3919
3920
3921
3922
3923
3924
3925
3926
3927
3928
3929
3930
3931
3932
3933
3934
3935
3936
3937
3938
3939
3940
3941
3942
3943
3944
3945
3946
3947
3948
3949
3950
3951
3952
3953
3954
3955
3956
3957
3958
3959
3960
3961
3962
3963
3964
3965
3966
3967
3968
3969
3970
3971
3972
3973
3974
3975
3976
3977
3978
3979
3980
3981
3982
3983
3984
3985
3986
3987
3988
3989
3990
3991
3992
3993
3994
3995
3996
3997


5796
5797
5798
5799
5800
5801
5802
5803
5804
5805
5806
5807
5808
5809
5810
5811
5812
5813
5814
5815
5816
5817
5818
5819
5820
5821
5822
5823
5824
5825
5826
5827
5828
5829
5830
5831
5832
5833
5834
5835
5836
5837
5838
5839
5840
5841
5842
5843
5844
5845
5846
5847
5848
5849
5850
5851
5852
5853
5854
5855
5856
5857
5858
5859
5860
5861
5862
5863
5864
5865
5866
5867
5868
5869
5870
5871
5872
5873
5874
5875
5876
5877
5878
5879
5880
5881
5882
5883
5884
5885
5886
5887
5888
5889
5890
5891
5892
5893
5894
5895
5896
5897
5898
5899
5900
5901
5902
5903
5904
5905
5906
5907
5908
5909
5910
5911
5912
5913
5914
5915
5916
5917
5918
5919
5920
5921
5922
5923
5924
5925
5926
5927
5928
5929
5930
5931
5932
5933
5934
5935
5936
5937
5938
5939
5940
5941
5942
5943
5944
5945
5946
5947
5948
5949
5950
5951
5952
5953
5954
5955
5956
5957
5958
5959
5960
5961
5962
5963
5964
5965
5966
5967
5968
5969
5970
5971
5972
5973
5974
5975
5976
5977
5978
5979
5980
5981
5982
5983
5984
5985
5986
5987
5988
5989
5990
5991
5992
5993
5994
5995


7794
7795
7796
7797
7798
7799
7800
7801
7802
7803
7804
7805
7806
7807
7808
7809
7810
7811
7812
7813
7814
7815
7816
7817
7818
7819
7820
7821
7822
7823
7824
7825
7826
7827
7828
7829
7830
7831
7832
7833
7834
7835
7836
7837
7838
7839
7840
7841
7842
7843
7844
7845
7846
7847
7848
7849
7850
7851
7852
7853
7854
7855
7856
7857
7858
7859
7860
7861
7862
7863
7864
7865
7866
7867
7868
7869
7870
7871
7872
7873
7874
7875
7876
7877
7878
7879
7880
7881
7882
7883
7884
7885
7886
7887
7888
7889
7890
7891
7892
7893
7894
7895
7896
7897
7898
7899
7900
7901
7902
7903
7904
7905
7906
7907
7908
7909
7910
7911
7912
7913
7914
7915
7916
7917
7918
7919
7920
7921
7922
7923
7924
7925
7926
7927
7928
7929
7930
7931
7932
7933
7934
7935
7936
7937
7938
7939
7940
7941
7942
7943
7944
7945
7946
7947
7948
7949
7950
7951
7952
7953
7954
7955
7956
7957
7958
7959
7960
7961
7962
7963
7964
7965
7966
7967
7968
7969
7970
7971
7972
7973
7974
7975
7976
7977
7978
7979
7980
7981
7982
7983
7984
7985
7986
7987
7988
7989
7990
7991
7992
7993


9792
9793
9794
9795
9796
9797
9798
9799
9800
9801
9802
9803
9804
9805
9806
9807
9808
9809
9810
9811
9812
9813
9814
9815
9816
9817
9818
9819
9820
9821
9822
9823
9824
9825
9826
9827
9828
9829
9830
9831
9832
9833
9834
9835
9836
9837
9838
9839
9840
9841
9842
9843
9844
9845
9846
9847
9848
9849
9850
9851
9852
9853
9854
9855
9856
9857
9858
9859
9860
9861
9862
9863
9864
9865
9866
9867
9868
9869
9870
9871
9872
9873
9874
9875
9876
9877
9878
9879
9880
9881
9882
9883
9884
9885
9886
9887
9888
9889
9890
9891
9892
9893
9894
9895
9896
9897
9898
9899
9900
9901
9902
9903
9904
9905
9906
9907
9908
9909
9910
9911
9912
9913
9914
9915
9916
9917
9918
9919
9920
9921
9922
9923
9924
9925
9926
9927
9928
9929
9930
9931
9932
9933
9934
9935
9936
9937
9938
9939
9940
9941
9942
9943
9944
9945
9946
9947
9948
9949
9950
9951
9952
9953
9954
9955
9956
9957
9958
9959
9960
9961
9962
9963
9964
9965
9966
9967
9968
9969
9970
9971
9972
9973
9974
9975
9976
9977
9978
9979
9980
9981
9982
9983
9984
9985
9986
9987
9988
9989
9990
9991


In [ ]:
loc_id = []
for i in range(1,len(df['Bien'])+1):
    print(i)
    id_bien.append(i)

In [322]:
bien_df

,Pièces,Taille,Prix,Description
0,4,125,386550.0,Maison gagny 4 pièces 125 m2. dans un secteur ...
1,2,39,284000.0,Appartement 2 pièces de 38m² environ à vendre...
2,3,69,399000.0,Appartement 3 pièces de 69 m² environ. votre ...
3,3,70,486000.0,Appartement le plessis robinson 3 pièces 70 m2...
4,3,61,330000.0,Appartement à vendre le plessis robinson 3 pi...
...,...,...,...,...
13494,4,86,294000.0,Appartement t4 proche gare a houilles. en plei...
13495,3,88,180000.0,Maison 3 pièces 88m² + studio indépendant proc...
13497,3,76,332000.0,T3 3 pièces. bel appartement de 3/4 pièces trè...
13498,3,74,199990.0,"T4 transforme en t3 spacieux, de 73m² + balcon..."


In [326]:
new_df.head()

,Bien,Pièces,Taille,Prix,Ville,Code Postal,Annonceur,Description,bien_id
0,Maison,4,125,386550.0,Gagny,93220,Professionnel,Maison gagny 4 pièces 125 m2. dans un secteur ...,1
1,Appartement,2,39,284000.0,Clamart,92140,Professionnel,Appartement 2 pièces de 38m² environ à vendre...,2
2,Appartement,3,69,399000.0,Clamart,92140,Professionnel,Appartement 3 pièces de 69 m² environ. votre ...,3
3,Appartement,3,70,486000.0,Le Plessis-Robinson,92350,Professionnel,Appartement le plessis robinson 3 pièces 70 m2...,4
4,Appartement,3,61,330000.0,Le Plessis-Robinson,92350,Professionnel,Appartement à vendre le plessis robinson 3 pi...,5


In [327]:
bien_df.head()

,Pièces,Taille,Prix,Description
0,4,125,386550.0,Maison gagny 4 pièces 125 m2. dans un secteur ...
1,2,39,284000.0,Appartement 2 pièces de 38m² environ à vendre...
2,3,69,399000.0,Appartement 3 pièces de 69 m² environ. votre ...
3,3,70,486000.0,Appartement le plessis robinson 3 pièces 70 m2...
4,3,61,330000.0,Appartement à vendre le plessis robinson 3 pi...


In [282]:
type_df.

type_bien        
Appartement          6066
Maison               4960
Duplex/triplex         76
Villa                  70
Propriété/château      34
Loft                   15
Chalet                  4
Atelier                 2
Ferme                   1
Moulin                  1
dtype: int64

In [290]:
req="select * from typebien"

In [291]:
df_req=pd.read_sql(req, conn)

In [375]:
df_req.head

<bound method NDFrame.head of     type_id          type_bien
0         1             Maison
1         2               Loft
2         3             Moulin
3         4  Propriété/château
4         5              Ferme
5         6              Villa
6         7        Appartement
7         8             Maison
8         9     Duplex/triplex
9        10             Chalet
10       11            Atelier>

In [308]:
req_tup = list(set([tuple(x) for x in df_req.to_numpy()]))
req_tup

[(9, 'Duplex/triplex'),
 (7, 'Appartement'),
 (8, 'Maison'),
 (6, 'Villa'),
 (11, 'Atelier'),
 (5, 'Ferme'),
 (1, 'Maison'),
 (2, 'Loft'),
 (10, 'Chalet'),
 (3, 'Moulin'),
 (4, 'Propriété/château')]

In [398]:
_new_df= new_df.copy()
_new_df.head()

,Bien,Pièces,Taille,Prix,Ville,Code Postal,Annonceur,Description
0,Appartement,3,63,348 000,Noisy-le-Grand,93160,Professionnel,Appartement noisy le grand 3 pièce(s) 63.17 m2...
1,Maison,8,156,315 000,Ablis,78660,Professionnel,Maison à 10' ablis sous sol 5 chambres. a 10' ...
2,Appartement,3,57,340 000,Alfortville,94140,Professionnel,A vendre appartement 3 pièces de 56.76 m² alf...
4,Maison,5,119,575 000,Fontenay-sous-Bois,94120,Professionnel,Maison fontenay sous bois 5 pièce(s) 118.50 m2...
5,Appartement,4,80,207 000,Chennevières-sur-Marne,94430,Professionnel,Appartement |chennevières sur marne | 3 chambr...


In [399]:
_new_df['Bien']=_new_df['Bien'].astype(str)

In [401]:
_new_df['Bien'].unique()

array(['Appartement', 'Maison', 'Villa', 'Duplex/triplex', 'Loft',
       'Propriété/château', 'Atelier', 'Chalet', 'Moulin', 'Ferme'],
      dtype=object)

In [402]:
dict_map={'Duplex/triplex':9,
  'Appartement':7,
 'Maison':8,
 'Villa':6,
 'Atelier':11,
 'Ferme':5,
 'Maison':1,
 'Loft':2,
 'Chalet':10,
 'Moulin':3,
  'Propriété/château':4}

In [403]:
dict_map

{'Duplex/triplex': 9,
 'Appartement': 7,
 'Maison': 1,
 'Villa': 6,
 'Atelier': 11,
 'Ferme': 5,
 'Loft': 2,
 'Chalet': 10,
 'Moulin': 3,
 'Propriété/château': 4}

In [404]:
_new_df['Bien']=_new_df['Bien'].map(dict_map)
_new_df

,Bien,Pièces,Taille,Prix,Ville,Code Postal,Annonceur,Description
0,7,3,63,348 000,Noisy-le-Grand,93160,Professionnel,Appartement noisy le grand 3 pièce(s) 63.17 m2...
1,1,8,156,315 000,Ablis,78660,Professionnel,Maison à 10' ablis sous sol 5 chambres. a 10' ...
2,7,3,57,340 000,Alfortville,94140,Professionnel,A vendre appartement 3 pièces de 56.76 m² alf...
4,1,5,119,575 000,Fontenay-sous-Bois,94120,Professionnel,Maison fontenay sous bois 5 pièce(s) 118.50 m2...
5,7,4,80,207 000,Chennevières-sur-Marne,94430,Professionnel,Appartement |chennevières sur marne | 3 chambr...
...,...,...,...,...,...,...,...,...
13495,7,3,64,460 000,Asnières-sur-Seine,92600,Professionnel,Vente appartement 3 pièces. iad france - samu...
13496,1,4,112,330 000,Le Blanc-Mesnil,93150,Professionnel,Vente maison/villa 4 pièces. iad france - son...
13497,1,3,84,286 000,Carrières-sous-Poissy,78955,Professionnel,Vente maison/villa 3 pièces. iad france - syl...
13498,7,3,63,425 000,Ivry-sur-Seine,94200,Professionnel,Vente appartement 3 pièces. iad france - chri...


In [405]:
_new_df.head()

,Bien,Pièces,Taille,Prix,Ville,Code Postal,Annonceur,Description
0,7,3,63,348 000,Noisy-le-Grand,93160,Professionnel,Appartement noisy le grand 3 pièce(s) 63.17 m2...
1,1,8,156,315 000,Ablis,78660,Professionnel,Maison à 10' ablis sous sol 5 chambres. a 10' ...
2,7,3,57,340 000,Alfortville,94140,Professionnel,A vendre appartement 3 pièces de 56.76 m² alf...
4,1,5,119,575 000,Fontenay-sous-Bois,94120,Professionnel,Maison fontenay sous bois 5 pièce(s) 118.50 m2...
5,7,4,80,207 000,Chennevières-sur-Marne,94430,Professionnel,Appartement |chennevières sur marne | 3 chambr...


In [406]:
_new_df['Bien'].astype(int)

0        7
1        1
2        7
4        1
5        7
        ..
13495    7
13496    1
13497    1
13498    7
13499    1
Name: Bien, Length: 11183, dtype: int32

In [307]:
bien_df.head()

,Pièces,Taille,Prix,Description
0,4,125,386550.0,Maison gagny 4 pièces 125 m2. dans un secteur ...
1,2,39,284000.0,Appartement 2 pièces de 38m² environ à vendre...
2,3,69,399000.0,Appartement 3 pièces de 69 m² environ. votre ...
3,3,70,486000.0,Appartement le plessis robinson 3 pièces 70 m2...
4,3,61,330000.0,Appartement à vendre le plessis robinson 3 pi...


In [302]:
bien_df.head()

,Pièces,Taille,Prix,Description
0,4,125,386550.0,Maison gagny 4 pièces 125 m2. dans un secteur ...
1,2,39,284000.0,Appartement 2 pièces de 38m² environ à vendre...
2,3,69,399000.0,Appartement 3 pièces de 69 m² environ. votre ...
3,3,70,486000.0,Appartement le plessis robinson 3 pièces 70 m2...
4,3,61,330000.0,Appartement à vendre le plessis robinson 3 pi...


In [305]:
merg_df=pd.merge(bien_df,df_req, left_index=True, right_index=True)

In [306]:
merg_df.head(100)

,Pièces,Taille,Prix,Description,type_id,type_bien
0,4,125,386550.0,Maison gagny 4 pièces 125 m2. dans un secteur ...,1,Maison
1,2,39,284000.0,Appartement 2 pièces de 38m² environ à vendre...,2,Loft
2,3,69,399000.0,Appartement 3 pièces de 69 m² environ. votre ...,3,Moulin
3,3,70,486000.0,Appartement le plessis robinson 3 pièces 70 m2...,4,Propriété/château
4,3,61,330000.0,Appartement à vendre le plessis robinson 3 pi...,5,Ferme
5,3,61,273500.0,Appartement à vendre à châtenay-malabry - 3/4 ...,6,Villa
6,3,64,388000.0,Appartement à vendre le plessis robinson 3 pi...,7,Appartement
7,5,106,499000.0,Appartement à vendre 5 pièces 106m² environ à ...,8,Maison
8,4,74,340000.0,Appartement à vendre à antony 4 pièces environ...,9,Duplex/triplex
9,3,62,329000.0,Appartement à vendre à antony 3 pièces d'envir...,10,Chalet


In [372]:
req1='''SELECT * from BIEN where price < 250000 and ville = paris'''

In [407]:
liste=_new_df['Ville'].unique().tolist()

In [408]:
liste

['Noisy-le-Grand',
 'Ablis',
 'Alfortville',
 'Fontenay-sous-Bois',
 'Chennevières-sur-Marne',
 "L'Haÿ-les-Roses",
 'Milly-la-Forêt',
 'Nanterre',
 'Issy-les-Moulineaux',
 'Montgeron',
 'Montry',
 'Bussy-Saint-Georges',
 'Serris',
 'Franconville',
 'Villabé',
 'Parmain',
 "L'Isle-Adam",
 'Persan',
 'Boulogne-Billancourt',
 'Levallois-Perret',
 'Épinay-sur-Seine',
 'Sevran',
 'Montigny-lès-Cormeilles',
 'Les Pavillons-sous-Bois',
 'Guyancourt',
 'Vincennes',
 'Mantes-la-Ville',
 'Houilles',
 'Villeneuve-le-Roi',
 'Bobigny',
 'Châtillon',
 'Ivry-sur-Seine',
 'Argenteuil',
 'Valenton',
 'La Plaine St Denis',
 'Bagnolet',
 'Mantes-la-Jolie',
 'Bondy',
 'Cergy',
 'Margency',
 'Sartrouville',
 'Versailles',
 'Courdimanche',
 'Le Chesnay',
 'Courbevoie',
 'Rungis',
 'Saulx-les-Chartreux',
 'Chevry-Cossigny',
 'Puiseux-Pontoise',
 'Saint-Michel-sur-Orge',
 'Le Bourget',
 'Clamart',
 'Villejust',
 'Brie-Comte-Robert',
 'Montreuil',
 'Champs-sur-Marne',
 'La Ferté-Alais',
 'Maisse',
 'Noisy-sur-

In [409]:
len(liste)

922

In [410]:
idlist=[]
for i in range(1,len(liste)+1):
    idlist.append(i)
len(idlist)
    

922

In [415]:
dict_ville={}
j=0
for n in liste:
    
    dict_ville[n]=idlist[j]
    j+=1

dict_ville

{'Noisy-le-Grand': 1,
 'Ablis': 2,
 'Alfortville': 3,
 'Fontenay-sous-Bois': 4,
 'Chennevières-sur-Marne': 5,
 "L'Haÿ-les-Roses": 6,
 'Milly-la-Forêt': 7,
 'Nanterre': 8,
 'Issy-les-Moulineaux': 9,
 'Montgeron': 10,
 'Montry': 11,
 'Bussy-Saint-Georges': 12,
 'Serris': 13,
 'Franconville': 14,
 'Villabé': 15,
 'Parmain': 16,
 "L'Isle-Adam": 17,
 'Persan': 18,
 'Boulogne-Billancourt': 19,
 'Levallois-Perret': 20,
 'Épinay-sur-Seine': 21,
 'Sevran': 22,
 'Montigny-lès-Cormeilles': 23,
 'Les Pavillons-sous-Bois': 24,
 'Guyancourt': 25,
 'Vincennes': 26,
 'Mantes-la-Ville': 27,
 'Houilles': 28,
 'Villeneuve-le-Roi': 29,
 'Bobigny': 30,
 'Châtillon': 31,
 'Ivry-sur-Seine': 32,
 'Argenteuil': 33,
 'Valenton': 34,
 'La Plaine St Denis': 35,
 'Bagnolet': 36,
 'Mantes-la-Jolie': 37,
 'Bondy': 38,
 'Cergy': 39,
 'Margency': 40,
 'Sartrouville': 41,
 'Versailles': 42,
 'Courdimanche': 43,
 'Le Chesnay': 44,
 'Courbevoie': 45,
 'Rungis': 46,
 'Saulx-les-Chartreux': 47,
 'Chevry-Cossigny': 48,
 'Pu

In [416]:
_new_df['Ville']=_new_df['Ville'].map(dict_ville)
_new_df

,Bien,Pièces,Taille,Prix,Ville,Code Postal,Annonceur,Description
0,7,3,63,348 000,1,93160,Professionnel,Appartement noisy le grand 3 pièce(s) 63.17 m2...
1,1,8,156,315 000,2,78660,Professionnel,Maison à 10' ablis sous sol 5 chambres. a 10' ...
2,7,3,57,340 000,3,94140,Professionnel,A vendre appartement 3 pièces de 56.76 m² alf...
4,1,5,119,575 000,4,94120,Professionnel,Maison fontenay sous bois 5 pièce(s) 118.50 m2...
5,7,4,80,207 000,5,94430,Professionnel,Appartement |chennevières sur marne | 3 chambr...
...,...,...,...,...,...,...,...,...
13495,7,3,64,460 000,183,92600,Professionnel,Vente appartement 3 pièces. iad france - samu...
13496,1,4,112,330 000,192,93150,Professionnel,Vente maison/villa 4 pièces. iad france - son...
13497,1,3,84,286 000,403,78955,Professionnel,Vente maison/villa 3 pièces. iad france - syl...
13498,7,3,63,425 000,32,94200,Professionnel,Vente appartement 3 pièces. iad france - chri...


In [417]:
_new_df["Annonceur"].unique()

array(['Professionnel', 'Particulier'], dtype=object)

In [418]:
dict_ann={'Professionnel':1, 'Particulier':2}

In [419]:
_new_df['Annoceur']=_new_df['Annonceur'].map(dict_ann)
_new_df

,Bien,Pièces,Taille,Prix,Ville,Code Postal,Annonceur,Description,Annoceur
0,7,3,63,348 000,1,93160,Professionnel,Appartement noisy le grand 3 pièce(s) 63.17 m2...,1
1,1,8,156,315 000,2,78660,Professionnel,Maison à 10' ablis sous sol 5 chambres. a 10' ...,1
2,7,3,57,340 000,3,94140,Professionnel,A vendre appartement 3 pièces de 56.76 m² alf...,1
4,1,5,119,575 000,4,94120,Professionnel,Maison fontenay sous bois 5 pièce(s) 118.50 m2...,1
5,7,4,80,207 000,5,94430,Professionnel,Appartement |chennevières sur marne | 3 chambr...,1
...,...,...,...,...,...,...,...,...,...
13495,7,3,64,460 000,183,92600,Professionnel,Vente appartement 3 pièces. iad france - samu...,1
13496,1,4,112,330 000,192,93150,Professionnel,Vente maison/villa 4 pièces. iad france - son...,1
13497,1,3,84,286 000,403,78955,Professionnel,Vente maison/villa 3 pièces. iad france - syl...,1
13498,7,3,63,425 000,32,94200,Professionnel,Vente appartement 3 pièces. iad france - chri...,1


In [421]:
new_df['type_id']=_new_df["Bien"]
new_df['ann_id']=_new_df["Annoceur"]
new_df['loc_id']=_new_df["Ville"]
new_df

,Bien,Pièces,Taille,Prix,Ville,Code Postal,Annonceur,Description,type_id
0,Appartement,3,63,348 000,Noisy-le-Grand,93160,Professionnel,Appartement noisy le grand 3 pièce(s) 63.17 m2...,7
1,Maison,8,156,315 000,Ablis,78660,Professionnel,Maison à 10' ablis sous sol 5 chambres. a 10' ...,1
2,Appartement,3,57,340 000,Alfortville,94140,Professionnel,A vendre appartement 3 pièces de 56.76 m² alf...,7
4,Maison,5,119,575 000,Fontenay-sous-Bois,94120,Professionnel,Maison fontenay sous bois 5 pièce(s) 118.50 m2...,1
5,Appartement,4,80,207 000,Chennevières-sur-Marne,94430,Professionnel,Appartement |chennevières sur marne | 3 chambr...,7
...,...,...,...,...,...,...,...,...,...
13495,Appartement,3,64,460 000,Asnières-sur-Seine,92600,Professionnel,Vente appartement 3 pièces. iad france - samu...,7
13496,Maison,4,112,330 000,Le Blanc-Mesnil,93150,Professionnel,Vente maison/villa 4 pièces. iad france - son...,1
13497,Maison,3,84,286 000,Carrières-sous-Poissy,78955,Professionnel,Vente maison/villa 3 pièces. iad france - syl...,1
13498,Appartement,3,63,425 000,Ivry-sur-Seine,94200,Professionnel,Vente appartement 3 pièces. iad france - chri...,7
